In [1]:
!pip install transformers==4.44.2 datasets==2.20.0 peft==0.12.0 torch==2.6.0 numpy==1.26.4 scikit-learn==1.5.2 nltk -q
!pip install torchvision==0.21.0 torchaudio==2.6.0 -q  # Match torch version
!pip install fsspec==2025.3.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 95.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import RobertaTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
import torch
import os

2025-05-12 09:06:36.391382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747040796.564210      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747040796.614021      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd

# Path to the uploaded dataset
dataset_path = '/kaggle/input/stm-32-clean-berasmi/stm32_clean.csv'
df = pd.read_csv(dataset_path)

In [4]:
os.environ["WANDB_DISABLED"] = "true"

formulating prompt (input) / code (output)

In [5]:
# Step 2
def format_data(row):
    input_text = f"Generate STM32 code for the following description: {row['comment']}"
    target_text = row['code']
    return pd.Series([input_text, target_text], index=['input_text', 'target_text'])

df_formatted = df.apply(format_data, axis=1)
df_formatted = df_formatted.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
df_formatted = df_formatted.dropna(subset=['input_text', 'target_text'])  # Remove NaN/None
df_formatted = df_formatted[df_formatted['input_text'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]
df_formatted = df_formatted[df_formatted['target_text'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]

train/test

In [7]:
# Step 3
train_df, temp_df = train_test_split(df_formatted, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [8]:
# Step 4: Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])
val_dataset = Dataset.from_pandas(val_df[['input_text', 'target_text']])
test_dataset = Dataset.from_pandas(test_df[['input_text', 'target_text']])

load model

In [9]:
# Step 5: Load Model and Tokenizer
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

sequence info

In [12]:
combined_df = pd.concat(
    [train_df[['input_text', 'target_text']],
     val_df[['input_text', 'target_text']],
     test_df[['input_text', 'target_text']]],
    ignore_index=True
)

# Convert to Hugging Face Dataset
combined_dataset = Dataset.from_pandas(combined_df)
def analyze_sequence_lengths(dataset, tokenizer):
    input_lengths = []
    target_lengths = []
    input_examples = []
    target_examples = []

    # Collect token lengths and examples
    for example in dataset:
        input_tokens = tokenizer(example['input_text'], truncation=False, padding=False)['input_ids']
        target_tokens = tokenizer(example['target_text'], truncation=False, padding=False)['input_ids']
        input_len = len(input_tokens)
        target_len = len(target_tokens)
        input_lengths.append(input_len)
        target_lengths.append(target_len)
        input_examples.append((input_len, example['input_text']))
        target_examples.append((target_len, example['target_text']))

    # Compute averages
    avg_input_length = sum(input_lengths) / len(input_lengths)
    avg_target_length = sum(target_lengths) / len(target_lengths)

    # Get top 5 longest sequences
    top_5_inputs = sorted(input_examples, key=lambda x: x[0], reverse=True)[:5]
    top_5_targets = sorted(target_examples, key=lambda x: x[0], reverse=True)[:5]

    # Print results
    print(f"\nAverage Sequence Lengths (in tokens):")
    print(f"Input (prompts): {avg_input_length:.2f} tokens")
    print(f"Target (code): {avg_target_length:.2f} tokens")

    print(f"\nTop 5 Longest Input Sequences:")
    for i, (length, text) in enumerate(top_5_inputs, 1):
        print(f"{i}. Length: {length} tokens, Text: {text}")

    print(f"\nTop 5 Longest Target Sequences:")
    for i, (length, text) in enumerate(top_5_targets, 1):
        print(f"{i}. Length: {length} tokens, Text: {text}")

# Run analysis
analyze_sequence_lengths(combined_dataset, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors



Average Sequence Lengths (in tokens):
Input (prompts): 20.57 tokens
Target (code): 45.14 tokens

Top 5 Longest Input Sequences:
1. Length: 103 tokens, Text: Generate STM32 code for the following description: Check the availability of adafruit 1.8" TFT shield on top of STM32NUCLEO board. This is done by reading the state of IO PF.03 pin (mapped to JoyStick available on adafruit 1.8" TFT shield). If the state of PF.03 is high then the adafruit 1.8" TFT shield is available.
2. Length: 103 tokens, Text: Generate STM32 code for the following description: Check the availability of adafruit 1.8" TFT shield on top of STM32NUCLEO board. This is done by reading the state of IO PF.03 pin (mapped to JoyStick available on adafruit 1.8" TFT shield). If the state of PF.03 is high then the adafruit 1.8" TFT shield is available.
3. Length: 83 tokens, Text: Generate STM32 code for the following description: STOP Mode Entry: RTC Clocked by LSI, Regulator in LP mode, HSI, HSE OFF and LSI OFF if not used 

sequences with simular length to have minimal padding

In [ ]:
def analyze_sequence_lengths(dataset, tokenizer, name="Dataset"):
    """
    Compute average and longest sequence lengths for input_ids and labels in a tokenized dataset.

    Args:
        dataset: Hugging Face Dataset object with tokenized 'input_ids' and 'labels'
        tokenizer: The tokenizer used (e.g., RobertaTokenizer for CodeT5)
        name: Name of the dataset (e.g., 'Train', 'Validation') for printing
    """
    # Extract sequence lengths for input_ids and labels
    input_lengths = [len(seq) for seq in dataset['input_ids']]
    label_lengths = [len(seq) for seq in dataset['labels']]

    # Compute average and max lengths
    avg_input_length = np.mean(input_lengths)
    max_input_length = np.max(input_lengths)
    avg_label_length = np.mean(label_lengths)
    max_label_length = np.max(label_lengths)

    # Find indices of the longest sequences
    max_input_idx = np.argmax(input_lengths)
    max_label_idx = np.argmax(label_lengths)

    # Decode the longest sequences for inspection
    longest_input_text = tokenizer.decode(dataset['input_ids'][max_input_idx], skip_special_tokens=True)
    longest_label_text = tokenizer.decode(dataset['labels'][max_label_idx], skip_special_tokens=True)

    # Print results
    print(f"\n{name} Sequence Length Analysis:")
    print(f"Input Sequences (input_ids):")
    print(f"  Average Length: {avg_input_length:.2f} tokens")
    print(f"  Max Length: {max_input_length} tokens")
    print(f"  Longest Input Text: {longest_input_text}")
    print(f"Label Sequences (labels):")
    print(f"  Average Length: {avg_label_length:.2f} tokens")
    print(f"  Max Length: {max_label_length} tokens")
    print(f"  Longest Label Text: {longest_label_text}")

# Analyze all datasets
analyze_sequence_lengths(train_dataset, tokenizer, name="Train")
analyze_sequence_lengths(val_dataset, tokenizer, name="Validation")
analyze_sequence_lengths(test_dataset, tokenizer, name="Test")

In [10]:
def filter_sequences_by_length(dataset, tokenizer, max_input_tokens=32, max_target_tokens=64):
    original_size = len(dataset)
    filtered_dataset = dataset.filter(
        lambda x: len(tokenizer(x['input_text'], truncation=False, padding=False)['input_ids']) <= max_input_tokens and
                  len(tokenizer(x['target_text'], truncation=False, padding=False)['input_ids']) <= max_target_tokens,
        desc="Filtering sequences"
    )
    filtered_size = len(filtered_dataset)
    print(f"Original dataset size: {original_size}")
    print(f"Filtered dataset size: {filtered_size}")
    print(f"Removed {original_size - filtered_size} sequences")
    return filtered_dataset

In [ ]:
filter_sequences_by_length(train_dataset,tokenizer)

In [11]:
train_dataset=filter_sequences_by_length(train_dataset,tokenizer)

Filtering sequences:   0%|          | 0/3847 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors


Original dataset size: 3847
Filtered dataset size: 3104
Removed 743 sequences


In [12]:
val_dataset=filter_sequences_by_length(val_dataset,tokenizer)

Filtering sequences:   0%|          | 0/481 [00:00<?, ? examples/s]

Original dataset size: 481
Filtered dataset size: 357
Removed 124 sequences


In [13]:
test_dataset=filter_sequences_by_length(test_dataset,tokenizer)

Filtering sequences:   0%|          | 0/481 [00:00<?, ? examples/s]

Original dataset size: 481
Filtered dataset size: 378
Removed 103 sequences


In [14]:
# Add a column for sequence length to the dataset
def add_sequence_length(example, tokenizer):
    input_tokens = tokenizer(example['input_text'], truncation=False, padding=False)['input_ids']
    target_tokens = tokenizer(example['target_text'], truncation=False, padding=False)['input_ids']
    return {
        'input_length': len(input_tokens),
        'target_length': len(target_tokens)
    }

# Apply length calculation to datasets
train_dataset = train_dataset.map(lambda x: add_sequence_length(x, tokenizer))
val_dataset = val_dataset.map(lambda x: add_sequence_length(x, tokenizer))
test_dataset = test_dataset.map(lambda x: add_sequence_length(x, tokenizer))



Map:   0%|          | 0/3104 [00:00<?, ? examples/s]

Map:   0%|          | 0/357 [00:00<?, ? examples/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

In [15]:
# Collect input lengths for sorting in collator
train_input_lengths = [example['input_length'] for example in train_dataset]
val_input_lengths = [example['input_length'] for example in val_dataset]
test_input_lengths = [example['input_length'] for example in test_dataset]

In [16]:
# Step 6: Tokenize the Dataset
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples['input_text'],
        #max_length=32,  # Reduced for Colab memory constraints
        truncation=True,
        padding='longest'
    )
    labels = tokenizer(
        examples['target_text'],
        #max_length=64,
        truncation=True,
        padding='longest'
    )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3104 [00:00<?, ? examples/s]

Map:   0%|          | 0/357 [00:00<?, ? examples/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

In [17]:
# Sort datasets by input_length
train_dataset = train_dataset.sort('input_length')
val_dataset = val_dataset.sort('input_length')
test_dataset = test_dataset.sort('input_length')

In [44]:
# Debug: Verify dataset keys
print("Train dataset keys:", train_dataset[0].keys())
print("Val dataset keys:", val_dataset[0].keys())
print("Test dataset keys:", test_dataset[0].keys())

Train dataset keys: dict_keys(['input_text', 'target_text', '__index_level_0__', 'input_length', 'target_length', 'input_ids', 'attention_mask', 'labels'])
Val dataset keys: dict_keys(['input_text', 'target_text', '__index_level_0__', 'input_length', 'target_length', 'input_ids', 'attention_mask', 'labels'])
Test dataset keys: dict_keys(['input_text', 'target_text', '__index_level_0__', 'input_length', 'target_length', 'input_ids', 'attention_mask', 'labels'])


In [18]:
# Set dataset format for PyTorch, retaining input_text and target_text
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'input_text', 'target_text'])

In [ ]:
# Step 7: Configure LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    #target_modules=["q", "v", "k", "o"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)

In [19]:
# Custom Data Collator
class LengthSortedDataCollator(DataCollatorForSeq2Seq):
    def __init__(self, tokenizer, model, max_batch_size, input_lengths, pad_to_multiple_of=8):
        super().__init__(tokenizer=tokenizer, model=model, pad_to_multiple_of=pad_to_multiple_of)
        self.max_batch_size = max_batch_size
        self.input_lengths = input_lengths  # Store input lengths

    def __call__(self, features):
        # Debug: Print available keys in features
        #print("Collator features keys:", [f.keys() for f in features])

        # Pair features with their corresponding input lengths
        # Since dataset is pre-sorted, we can use the order of input_lengths
        indexed_features = list(zip(self.input_lengths[:len(features)], features))

        # Sort by input_length (descending)
        indexed_features = sorted(indexed_features, key=lambda x: x[0], reverse=True)

        # Extract sorted features
        sorted_features = [f for _, f in indexed_features]

        # Group into batches of max_batch_size
        batched_features = []
        for i in range(0, len(sorted_features), self.max_batch_size):
            batch = sorted_features[i:i + self.max_batch_size]
            batched_features.append(batch)

        # Process each batch using the parent class's collation logic
        final_batches = []
        for batch in batched_features:
            batch = super().__call__(batch)
            final_batches.append(batch)

        # Return the first batch (Trainer expects a single batch at a time)
        return final_batches[0]

In [20]:
# Step 8: Set Up Data Collator
data_collator = LengthSortedDataCollator(
    tokenizer=tokenizer,
    model=model,
    max_batch_size=2,  # Match per_device_train_batch_size
    input_lengths=train_input_lengths,  # Pass train input lengths
    pad_to_multiple_of=8
)

In [ ]:
# Step 9: Configure Training Arguments
training_args = TrainingArguments(
    output_dir="./stm32_codet5",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Match max_batch_size in collator
    per_device_eval_batch_size=2,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    report_to="none",
    run_name="stm32_codet5_run",
    fp16=True,
)

Xai: metrics / attention score / how each hyperparameters affects loss.. (not actual training)

In [ ]:
import os
import json
from transformers import Trainer, TrainingArguments, T5ForConditionalGeneration
from peft import LoraConfig, get_peft_model
from itertools import product

# Default hyperparameter values (based on your original code)
default_r = 32
default_target_modules = ["q", "v", "k", "o"]
default_learning_rate = 2e-5
default_batch_size = 2

# Hyperparameter ranges to test
hyperparam_ranges = {
    'r': [8, 16, 32, 64],
    'target_modules': [
        ["q", "v"],
        ["q", "v", "k", "o"],
        ["q", "v", "k", "o", "wi", "wo"]
    ],
    'learning_rate': [1e-5, 2e-5, 5e-5],
    'per_device_train_batch_size': [2, 4]
}

# Function to configure and train model for a single hyperparameter setting
def train_with_hyperparam(hyperparam_name, hyperparam_value, output_dir):
    # Configure LoRA with the tested hyperparameter
    lora_config_params = {
        'r': hyperparam_value if hyperparam_name == 'r' else default_r,
        'lora_alpha': 32,
        'target_modules': hyperparam_value if hyperparam_name == 'target_modules' else default_target_modules,
        'lora_dropout': 0.1,
        'bias': "none",
        'task_type': "SEQ_2_SEQ_LM"
    }
    lora_config = LoraConfig(**lora_config_params)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model = get_peft_model(model, lora_config)

    # Configure batch size
    batch_size = hyperparam_value if hyperparam_name == 'per_device_train_batch_size' else default_batch_size

    # Update data collator with batch size
    data_collator = LengthSortedDataCollator(
        tokenizer=tokenizer,
        model=model,
        max_batch_size=batch_size,
        input_lengths=train_input_lengths,
        pad_to_multiple_of=8
    )

    # Configure training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        learning_rate=hyperparam_value if hyperparam_name == 'learning_rate' else default_learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=2,
        weight_decay=0.01,
        logging_dir=f'{output_dir}/logs',
        logging_steps=10,
        save_total_limit=1,
        report_to="none",
        run_name=f"stm32_codet5_{hyperparam_name}_{str(hyperparam_value).replace('.', '_')}",
        fp16=True,
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

    # Train and evaluate
    trainer.train()
    eval_results = trainer.evaluate()

    # Save loss logs
    log_history = trainer.state.log_history
    with open(f"{output_dir}/loss_log.json", "w") as f:
        json.dump(log_history, f, indent=4)

    print(f"Completed: {hyperparam_name}={hyperparam_value}, Validation Loss: {eval_results['eval_loss']}")

# Run experiments for each hyperparameter
for hyperparam_name, values in hyperparam_ranges.items():
    print(f"\nTesting hyperparameter: {hyperparam_name}")
    for value in values:
        output_dir = f"./stm32_codet5_{hyperparam_name}_{str(value).replace('.', '_')}"
        try:
            train_with_hyperparam(hyperparam_name, value, output_dir)
        except Exception as e:
            print(f"Failed for {hyperparam_name}={value}: {e}")

# After running, analyze logs
print("\nTo analyze results, check the 'loss_log.json' files in each output directory.")
print("Each file contains training and validation loss per logging step/epoch.")
print("Use the following script to visualize loss trends:")

# Script to analyze and visualize loss trends
import matplotlib.pyplot as plt
import json
import os

def plot_loss_trends(hyperparam_name, values, base_dir="./"):
    plt.figure(figsize=(10, 6))
    for value in values:
        output_dir = f"{base_dir}/stm32_codet5_{hyperparam_name}_{str(value).replace('.', '_')}"
        log_file = f"{output_dir}/loss_log.json"
        if os.path.exists(log_file):
            with open(log_file, "r") as f:
                logs = json.load(f)
            train_loss = [(log['step'], log['loss']) for log in logs if 'loss' in log]
            eval_loss = [(log['epoch'], log['eval_loss']) for log in logs if 'eval_loss' in log]
            
            if train_loss:
                steps, losses = zip(*train_loss)
                plt.plot(steps, losses, label=f"{hyperparam_name}={value} (Train)")
            if eval_loss:
                epochs, losses = zip(*eval_loss)
                plt.plot(epochs, losses, '--', label=f"{hyperparam_name}={value} (Val)")
    
    plt.title(f"Loss Trends for {hyperparam_name}")
    plt.xlabel("Steps (Train) / Epochs (Val)")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{base_dir}/loss_trends_{hyperparam_name}.png")
    plt.close()

# Generate plots for each hyperparameter
for hyperparam_name, values in hyperparam_ranges.items():
    plot_loss_trends(hyperparam_name, values)


Testing hyperparameter: r


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss


execute for actual training

In [28]:
# Step 10: Initialize and Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.938400,0.820758
2,0.818300,0.744302
3,0.861400,0.705085
4,0.725000,0.683939
5,0.717100,0.670061
6,0.546800,0.662210
7,0.746300,0.659903


TrainOutput(global_step=10864, training_loss=0.8474417003818043, metrics={'train_runtime': 1702.0133, 'train_samples_per_second': 12.766, 'train_steps_per_second': 6.383, 'total_flos': 841728497025024.0, 'train_loss': 0.8474417003818043, 'epoch': 7.0})

test prompts different prompts (short, long)

In [24]:
# Define prompts
prompts = {
    'concise': lambda comment: f"Generate STM32 code for: {comment}",
    'detailed': lambda comment: f"As an STM32 programming expert, generate the STM32 code that implements the functionality described: {comment}",
    'structured': lambda comment: f"[Task: Generate STM32 Code]\n[Comment]: {comment}\n[Output]: STM32 code",
    'action_oriented': lambda comment: f"Write STM32 code to: {comment}"
}

# Test comments
test_comments = [
    "Function call for delay of 500ms",
    "Toggle LED9 on the STM32 board",
    "Reset the LED pin to turn it off",
    "Initialize UART4 peripheral"
]

# Verify model and tokenizer are in memory
try:
    assert 'model' in globals() and 'tokenizer' in globals(), "Model or tokenizer not found in memory"
    print("Model and tokenizer found in memory!")
except AssertionError:
    raise Exception("Model or tokenizer not found. Ensure you run this in the same session as training or load the saved model.")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Generate outputs for each prompt and comment
for prompt_name, prompt_fn in prompts.items():
    print(f"\n=== Outputs for {prompt_name} prompt ===\n")
    for comment in test_comments:
        input_text = prompt_fn(comment)
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        outputs = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Comment: {comment}")
        print(f"Generated Code: {generated_code}\n")

# Optional: Reload dataset if df is not in memory (uncomment if needed)
"""
import pandas as pd
df = pd.read_csv('stm32_code_comments_dataset_scored.csv')
df = df[df['relevance_score'] > 60]
print(f"Filtered dataset size: {len(df)}")
print(df[['comment', 'code']].head())
"""

Model and tokenizer found in memory!

=== Outputs for concise prompt ===

Comment: Function call for delay of 500ms
Generated Code: HAL_Delay(500);

Comment: Toggle LED9 on the STM32 board
Generated Code: HAL_LED9_Toggle(LED9);

Comment: Reset the LED pin to turn it off
Generated Code: HAL_LED_Pin_Reset();

Comment: Initialize UART4 peripheral
Generated Code: HAL_UART4_Init();


=== Outputs for detailed prompt ===

Comment: Function call for delay of 500ms
Generated Code: HAL_RCC_Delay(500);

Comment: Toggle LED9 on the STM32 board
Generated Code: HAL_LED9_Toggle(LED9);

Comment: Reset the LED pin to turn it off
Generated Code: HAL_LED_Pin_Reset(LED_Pin);

Comment: Initialize UART4 peripheral
Generated Code: HAL_UART4_Init();


=== Outputs for structured prompt ===

Comment: Function call for delay of 500ms
Generated Code: HAL_HAL_HAL_HAL_DELAY(500ms);

Comment: Toggle LED9 on the STM32 board
Generated Code: HAL_LED9_Toggle(LED9);

Comment: Reset the LED pin to turn it off
Generated Co

'\nimport pandas as pd\ndf = pd.read_csv(\'stm32_code_comments_dataset_scored.csv\')\ndf = df[df[\'relevance_score\'] > 60]\nprint(f"Filtered dataset size: {len(df)}")\nprint(df[[\'comment\', \'code\']].head())\n'

In [ ]:
import random

In [ ]:
def display_random_tokens(dataset, tokenizer, num_examples=5):
    random_indices = random.sample(range(len(dataset)), num_examples)
    print(f"\nDisplaying tokens for {num_examples} random sequences:\n")
    for i, idx in enumerate(random_indices):
        input_ids = dataset[idx]['input_ids'].tolist()
        labels = dataset[idx]['labels'].tolist()
        input_tokens = [tokenizer.convert_ids_to_tokens(id) for id in input_ids]
        label_tokens = [tokenizer.convert_ids_to_tokens(id) for id in labels]
        print(f"Example {i+1} (Index: {idx}):")
        print(f"Input Tokens: {input_tokens}")
        print(f"Label Tokens: {label_tokens}")
        print("-" * 80)

display_random_tokens(train_dataset, tokenizer, num_examples=5)

test

In [29]:
pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
import torch
import numpy as np
from captum.attr import IntegratedGradients
import shap
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import T5ForConditionalGeneration, RobertaTokenizer
from datasets import Dataset


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

In [27]:
sample = test_dataset[0]
input_text = sample['input_text']
target_text = sample['target_text']  # or whatever the actual key name is

# 1. Attention Visualization
def visualize_attention(input_text, output_text, model, tokenizer, layer_idx=0, head_idx=0):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=64, output_attentions=True, return_dict_in_generate=True)
    
    # Extract cross-attention (decoder attending to encoder outputs) from the first layer and head
    cross_attention = outputs.attentions[0][layer_idx][0][head_idx]  # [tgt_len, src_len]
    
    input_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    output_tokens = tokenizer.convert_ids_to_tokens(outputs.sequences[0])
    
    # Plot heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(cross_attention.cpu().detach().numpy(), xticklabels=input_tokens, yticklabels=output_tokens, cmap="viridis")
    plt.title(f"Cross-Attention (Layer {layer_idx}, Head {head_idx})")
    plt.xlabel("Input Tokens")
    plt.ylabel("Output Tokens")
    plt.savefig("attention_heatmap.png")
    plt.close()


visualize_attention(input_text, target_text, model, tokenizer, layer_idx=0, head_idx=0)


AttributeError: 'GenerateEncoderDecoderOutput' object has no attribute 'attentions'

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Input prompt
prompt = "[Instruction] Generate STM32 code for the following description: initialize GPIO for STM32"
inputs = tokenizer(prompt, return_tensors="pt", max_length=32, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}  # MOVE INPUTS TO DEVICE

model = model.to(device)  # MOVE MODEL TO DEVICE

# Forward pass with attention outputs
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [29]:
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
decoder_inputs = tokenizer(target_text, return_tensors="pt")

inputs = {k: v.to(device) for k, v in inputs.items()}
decoder_inputs = {k: v.to(device) for k, v in decoder_inputs.items()}
model.to(device)


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32100, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32100, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
             

In [38]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu
import re

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define 10 prompts and target STM32 code snippets
prompts_targets = [
    {
        "prompt": "Generate STM32 code to enable the clock for GPIO Port B.",
        "target": "__HAL_RCC_GPIOB_CLK_ENABLE();"
    },
    {
        "prompt": "Generate STM32 code to initialize a delay with 80 MHz system clock.",
        "target": "delay_init(80);"
    },
    {
        "prompt": "Generate STM32 code to transmit data over UART1 with HAL.",
        "target": "HAL_UART_Transmit(&huart1, data, size, 1000);"
    },
    {
        "prompt": "Generate STM32 code to initialize the system kernel.",
        "target": "osKernelStart();"
    },
    {
        "prompt": "Generate STM32 code to initialize a PID controller with float32.",
        "target": "arm_pid_init_f32(&pid, 1);"
    },
    {
        "prompt": "Generate STM32 code to enable the clock for GPIO Port A.",
        "target": "__HAL_RCC_GPIOA_CLK_ENABLE();"
    },
    {
        "prompt": "Generate STM32 code to configure GPIO Pin 5 as output.",
        "target": "HAL_GPIO_Init(GPIOA, &GPIO_InitStruct);"
    },
    {
        "prompt": "Generate STM32 code to toggle GPIO Pin 13 on Port C.",
        "target": "HAL_GPIO_TogglePin(GPIOC, GPIO_PIN_13);"
    },
    {
        "prompt": "Generate STM32 code to initialize the network stack.",
        "target": "net_init();"
    },
    {
        "prompt": "Generate STM32 code to start the ADC conversion.",
        "target": "HAL_ADC_Start(&hadc1);"
    }
]

def clean_tokens(tokens):
    """Remove special tokens and punctuation, keep meaningful code tokens."""
    # Define tokens to exclude (special tokens and punctuation)
    exclude = {'<s>', '</s>', '(', ')', ';', '&', ','}
    # Keep only alphanumeric tokens or underscores, exclude special/punctuation
    return [t for t in tokens if t not in exclude and re.match(r'[\w]+', t)]

def evaluate_token_accuracy(prompt, target, model, tokenizer):
    """Generate prediction and compute token-based accuracy ignoring order."""
    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    
    # Generate prediction
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=64,
        num_beams=5,
        early_stopping=True
    )
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Tokenize target and predicted text
    target_tokens = tokenizer.tokenize(target)
    predicted_tokens = tokenizer.tokenize(predicted_text)
    
    # Clean tokens (remove punctuation, special tokens)
    target_tokens_clean = clean_tokens(target_tokens)
    predicted_tokens_clean = clean_tokens(predicted_tokens)
    
    # Convert to sets to ignore order
    target_set = set(target_tokens_clean)
    predicted_set = set(predicted_tokens_clean)
    
    # Compute matching tokens
    matching_tokens = target_set & predicted_set
    num_matching = len(matching_tokens)
    num_target = len(target_set) if len(target_set) > 0 else 1  # Avoid division by zero
    
    # Accuracy for this example
    accuracy = num_matching / num_target
    
    return {
        "prompt": prompt,
        "target": target,
        "predicted": predicted_text,
        "target_tokens": target_tokens_clean,
        "predicted_tokens": predicted_tokens_clean,
        "matching_tokens": matching_tokens,
        "accuracy": accuracy
    }

# Evaluate all prompts
results = []
for item in prompts_targets:
    result = evaluate_token_accuracy(item["prompt"], item["target"], model, tokenizer)
    results.append(result)

# Compute overall accuracy
total_accuracy = sum(r["accuracy"] for r in results) / len(results)

# Print results
print("STM32 Code Prediction Results:")
for i, result in enumerate(results):
    print(f"\nExample {i+1}:")
    print(f"Prompt: {result['prompt']}")
    print(f"Target: {result['target']}")
    print(f"Predicted: {result['predicted']}")
    print(f"Target Tokens: {result['target_tokens']}")
    print(f"Predicted Tokens: {result['predicted_tokens']}")
    print(f"Matching Tokens: {result['matching_tokens']}")
    print(f"Accuracy: {result['accuracy']:.4f}")

print(f"\nOverall Token Accuracy: {total_accuracy:.4f}")

STM32 Code Prediction Results:

Example 1:
Prompt: Generate STM32 code to enable the clock for GPIO Port B.
Target: __HAL_RCC_GPIOB_CLK_ENABLE();
Predicted: HAL_GPIO_Init(GPIO_PORT_B);
Target Tokens: ['__', 'H', 'AL', '_', 'R', 'CC', '_', 'G', 'PI', 'OB', '_', 'CL', 'K', '_', 'ENABLE']
Predicted Tokens: ['H', 'AL', '_', 'G', 'PIO', '_', 'Init', 'G', 'PIO', '_', 'PORT', '_', 'B']
Matching Tokens: {'H', 'G', 'AL', '_'}
Accuracy: 0.3333

Example 2:
Prompt: Generate STM32 code to initialize a delay with 80 MHz system clock.
Target: delay_init(80);
Predicted: HAL_Delay_Init(80);
Target Tokens: ['delay', '_', 'init', '80']
Predicted Tokens: ['H', 'AL', '_', 'Delay', '_', 'Init', '80']
Matching Tokens: {'80', '_'}
Accuracy: 0.5000

Example 3:
Prompt: Generate STM32 code to transmit data over UART1 with HAL.
Target: HAL_UART_Transmit(&huart1, data, size, 1000);
Predicted: HAL_UART1_Transmit(&huart1, 1);
Target Tokens: ['H', 'AL', '_', 'U', 'ART', '_', 'Trans', 'mit', 'h', 'u', 'art', '1', 'Ġda

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from peft import PeftModel
import nltk
from nltk.metrics.distance import edit_distance
import torch
from nltk.tokenize import word_tokenize

# Download tokenizer data
nltk.download('punkt')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model (make sure these are defined before this block)
# tokenizer = ...
# model = ...
model = model.to(device)

# Original prompt
prompt = "[Instruction] Generate STM32 code for the following description: initialize GPIO for STM32"
original_inputs = tokenizer(prompt, return_tensors="pt", max_length=32, truncation=True)
original_inputs = {k: v.to(device) for k, v in original_inputs.items()}  # MOVE BEFORE GENERATE

with torch.no_grad():
    original_outputs = model.generate(**original_inputs, max_length=100, num_beams=3, no_repeat_ngram_size=2, early_stopping=True)
original_code = tokenizer.decode(original_outputs[0], skip_special_tokens=True)
print("Original code:", original_code)

# Tokenize prompt for perturbation
prompt_words = word_tokenize(prompt)
print("Prompt words:", prompt_words)

# Perturb by removing one word at a time
perturbed_codes = []
for i in range(len(prompt_words)):
    perturbed_words = prompt_words[:i] + prompt_words[i+1:]
    perturbed_prompt = " ".join(perturbed_words)
    inputs = tokenizer(perturbed_prompt, return_tensors="pt", max_length=32, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100, num_beams=3, no_repeat_ngram_size=2, early_stopping=True)
    perturbed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    edit_dist = edit_distance(original_code, perturbed_code)
    perturbed_codes.append((perturbed_prompt, perturbed_code, edit_dist))

# Print results
print("\nPerturbation Analysis:")
for perturbed_prompt, perturbed_code, edit_dist in perturbed_codes:
    print(f"Perturbed Prompt: {perturbed_prompt}")
    print(f"Perturbed Code: {perturbed_code}")
    print(f"Edit Distance: {edit_dist}\n")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from captum.attr import IntegratedGradients
import torch

# Load model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input prompt
prompt = "[Instruction] Generate STM32 code for the following description: initialize GPIO for STM32"
inputs = tokenizer(prompt, return_tensors="pt", max_length=32, truncation=True)

# Debugging: Check tensor type right after tokenization
print(f"input_ids dtype before transfer: {inputs['input_ids'].dtype}")

# Move inputs to device and cast to long (if needed)
input_ids = inputs["input_ids"].to(device).long()  # Ensure it's long dtype
attention_mask = inputs["attention_mask"].to(device).long()  # Same for attention_mask

# Debugging: Check tensor type after transfer and cast
print(f"input_ids dtype after transfer: {input_ids.dtype}")
print(f"attention_mask dtype after transfer: {attention_mask.dtype}")

# Forward function for Integrated Gradients
def forward_func(input_ids, attention_mask):
    # Debugging: Check tensor types inside forward pass
    print(f"Inside forward_func - input_ids dtype: {input_ids.dtype}, attention_mask dtype: {attention_mask.dtype}")
    
    # Ensure correct tensor types in forward pass
    assert input_ids.dtype == torch.long, f"Expected input_ids to be of type torch.long, but got {input_ids.dtype}"
    assert attention_mask.dtype == torch.long, f"Expected attention_mask to be of type torch.long, but got {attention_mask.dtype}"
    
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    return outputs.logits

# Integrated Gradients attribution
ig = IntegratedGradients(forward_func)

# Compute attributions
attributions, delta = ig.attribute(
    inputs=input_ids,
    additional_forward_args=(attention_mask,),
    target=0,  # Attribution for first token
    return_convergence_delta=True
)

# Sum attributions across embedding dimensions
attributions = attributions.sum(dim=-1).squeeze(0).cpu().detach().numpy()
input_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# Print token importance
print("Token Importance:")
for token, score in zip(input_tokens, attributions):
    print(f"Token: {token}, Score: {score:.4f}")

In [31]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install sentence_bleu

ERROR: Could not find a version that satisfies the requirement sentence_bleu (from versions: none)
ERROR: No matching distribution found for sentence_bleu
Note: you may need to restart the kernel to use updated packages.


In [32]:
import shap
from nltk.translate.bleu_score import sentence_bleu

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.config.output_attentions=True
sample = test_dataset[0]

# 1. Attention Visualization
def visualize_attention(input_text, target_text, model, tokenizer, layer_idx=0, head_idx=0):
    # Ensure model is configured to output attentions
    model.config.output_attentions = True
    
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Use model.generate to get attentions
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        output_attentions=True,
        return_dict_in_generate=True,
        max_length=64
    )
    
    # Debug: Print output keys and attentions structure
    print("Model output keys:", outputs.keys())
    if "cross_attentions" in outputs and outputs.cross_attentions is not None:
        cross_attention_tuple = outputs.cross_attentions[layer_idx]
        print("Cross attention tuple type:", type(cross_attention_tuple))
        if isinstance(cross_attention_tuple, tuple):
            print("Cross attention tuple length:", len(cross_attention_tuple))
            print("First element type:", type(cross_attention_tuple[0]))
            if hasattr(cross_attention_tuple[0], "shape"):
                print("First element shape:", cross_attention_tuple[0].shape)
            cross_attention = cross_attention_tuple[0][0][head_idx]
            print("Extracted cross_attention shape:", cross_attention.shape)
        else:
            print("Cross attention tensor shape:", cross_attention_tuple.shape)
            ross_attention = cross_attention_tuple[0][head_idx]
            print("Extracted cross_attention shape:", cross_attention.shape)
    
    # Get input and output tokens
    input_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    output_tokens = tokenizer.convert_ids_to_tokens(outputs.sequences[0])
    
    # Plot heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(cross_attention.cpu().detach().numpy(), xticklabels=input_tokens, yticklabels=output_tokens, cmap="viridis")
    plt.title(f"Cross-Attention (Layer {layer_idx}, Head {head_idx})")
    plt.xlabel("Input Tokens")
    plt.ylabel("Output Tokens")
    plt.savefig("attention_heatmap.png")
    plt.close()
    
    # Metric: Attention Entropy
    attention_probs = cross_attention.cpu().detach().numpy()
    entropy = -np.sum(attention_probs * np.log(attention_probs + 1e-10), axis=1).mean()
    return {"attention_entropy": entropy}



#visualize_attention(input_text, target_text, model, tokenizer, layer_idx=0, head_idx=0)

# 2. Integrated Gradients
def integrated_gradients_analysis(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
    input_ids = inputs["input_ids"]
    embeddings = model.get_input_embeddings()(input_ids)
    
    print("Input IDs shape:", input_ids.shape)
    print("Embeddings shape:", embeddings.shape)
    print("Labels shape:", inputs["input_ids"].shape)
    
    def forward_func(embeddings):
        # Provide decoder_input_ids explicitly
        outputs = model(
            inputs_embeds=embeddings,
            decoder_input_ids=inputs["input_ids"],  # Use input_ids as decoder input
            output_attentions=True
        )
        return outputs.logits[:, -1, :].softmax(dim=-1)
    
    from captum.attr import IntegratedGradients
    ig = IntegratedGradients(forward_func)
    baseline = torch.zeros_like(embeddings)
    attributions, delta = ig.attribute(embeddings, baseline, target=0, return_convergence_delta=True)
    
    # Debug: Print shapes
    print("Attributions shape:", attributions.shape)
    print("After sum(dim=-1):", attributions.sum(dim=-1).shape)
    print("After squeeze():", attributions.sum(dim=-1).squeeze().shape)
    
    # Sum attributions across embedding dimensions and ensure 1D output
    attribution_scores = attributions.sum(dim=-1).squeeze().cpu().detach().numpy()
    if attribution_scores.ndim > 1:
        attribution_scores = attribution_scores.flatten()
    
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    completeness = abs(delta.cpu().detach().numpy().sum())
    
    return {"tokens": input_tokens, "attribution_scores": attribution_scores, "completeness": completeness}

# 3. SHAP Analysis
def shap_analysis(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    def model_predict(texts):
        tokenized = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**tokenized,output_attentions=True)
        return outputs.logits.softmax(dim=-1).cpu().numpy()
    
    explainer = shap.KernelExplainer(model_predict, [tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)])
    shap_values = explainer.shap_values(input_text)
    
    # Metric: Mean Absolute SHAP
    mean_abs_shap = np.abs(shap_values).mean()
    
    return {"shap_values": shap_values, "mean_abs_shap": mean_abs_shap}

# 4. Perturbation Analysis
def perturbation_analysis(input_text, target_text, model, tokenizer):
    input_tokens = input_text.split()
    results = []
    
    for i, token in enumerate(input_tokens):
        # Remove one token
        perturbed_input = " ".join(input_tokens[:i] + input_tokens[i+1:])
        perturbed_inputs = tokenizer(perturbed_input, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            perturbed_output = model.generate(**perturbed_inputs, max_length=64)
        perturbed_code = tokenizer.decode(perturbed_output[0], skip_special_tokens=True)
        
        # BLEU Score
        reference = target_text.split()
        candidate = perturbed_code.split()
        bleu = sentence_bleu([reference], candidate)
        
        # Domain-Specific Check (Placeholder: Validate STM32 syntax)
        is_valid = True  # Replace with actual C compiler check for STM32
        results.append({"token_removed": token, "bleu_score": bleu, "valid_code": is_valid})
    
    return results

# 5. Qualitative Case Study
def qualitative_case_study(test_dataset, model, tokenizer, num_examples=5):
    examples = test_dataset.select(range(min(num_examples, len(test_dataset))))
    analysis_results = []
    
    for example in examples:
        input_text = example["input_text"]
        target_text = example["target_text"]
        
        # Run all analyses
        attention_metrics = visualize_attention(input_text, target_text, model, tokenizer)
        #ig_metrics = integrated_gradients_analysis(input_text, model, tokenizer)
        #shap_metrics = shap_analysis(input_text, model, tokenizer)
        perturbation_metrics = perturbation_analysis(input_text, target_text, model, tokenizer)
        
        result = {
            "input_text": example["input_text"],
            "target_text": example["target_text"],
            "attention_entropy": attention_metrics["attention_entropy"],
            #"ig_attributions": ig_metrics["attribution_scores"],
            #"ig_completeness": ig_metrics["completeness"],
            #"shap_mean_abs": shap_metrics["mean_abs_shap"],
            "perturbation_results": perturbation_metrics
        }
        
        # Debug: Print types and shapes
        print("Result dictionary:")
        for key, value in result.items():
            print(f"{key}: type={type(value)}", end="")
            if isinstance(value, np.ndarray):
                print(f", shape={value.shape}")
            elif isinstance(value, list):
                print(f", length={len(value)}")
                if value and isinstance(value[0], dict):
                    print(f"  First item keys: {list(value[0].keys())}")
            else:
                print()
        
        analysis_results.append(result)
    
    # Convert to DataFrame and save
    import pandas as pd
    df = pd.DataFrame(analysis_results)
    print("DataFrame columns:", df.columns)
    print("DataFrame head:\n", df.head())
    df.to_csv("xai_analysis_results.csv", index=False)
    return analysis_results

# Run XAI Analysis
xai_results = qualitative_case_study(test_dataset, model, tokenizer, num_examples=5)
print("XAI Analysis Completed. Results saved to 'xai_analysis_results.csv'.")

Model output keys: odict_keys(['sequences', 'encoder_attentions', 'decoder_attentions', 'cross_attentions', 'past_key_values'])
Cross attention tuple type: <class 'tuple'>
Cross attention tuple length: 12
First element type: <class 'torch.Tensor'>
First element shape: torch.Size([1, 12, 1, 14])
Extracted cross_attention shape: torch.Size([1, 14])
Result dictionary:
input_text: type=<class 'str'>
target_text: type=<class 'str'>
attention_entropy: type=<class 'numpy.float32'>
perturbation_results: type=<class 'list'>, length=9
  First item keys: ['token_removed', 'bleu_score', 'valid_code']
Model output keys: odict_keys(['sequences', 'encoder_attentions', 'decoder_attentions', 'cross_attentions', 'past_key_values'])
Cross attention tuple type: <class 'tuple'>
Cross attention tuple length: 12
First element type: <class 'torch.Tensor'>
First element shape: torch.Size([1, 12, 1, 14])
Extracted cross_attention shape: torch.Size([1, 14])
Result dictionary:
input_text: type=<class 'str'>
targe

In [34]:
!zip xai_analysis_results.zip xai_analysis_results.csv

  adding: xai_analysis_results.csv (deflated 88%)


In [ ]:
print("Output attentions enabled:", model.config.output_attentions)

In [ ]:
    # Metric: Attention Entropy
    attention_probs = cross_attention.cpu().detach().numpy()
    entropy = -np.sum(attention_probs * np.log(attention_probs + 1e-10), axis=1).mean()
    return {"attention_entropy": entropy}

In [ ]:
pip install lime

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer
import torch
import numpy as np

# Define a prediction function for LIME
def predict_proba(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate code (get logits)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu().numpy()

    # Convert logits to probabilities (simplified for LIME)
    probs = np.softmax(logits, axis=-1)
    return probs

# Initialize LIME explainer
explainer = LimeTextExplainer(class_names=["code_output"])  # Simplified for single output

# Select a test comment
test_comment = test_dataset[0]["input_text"]  # e.g., "[Instruction] Generate STM32 code for GPIO toggle"

# Generate LIME explanation
explanation = explainer.explain_instance(test_comment, predict_proba, num_features=10)

# Display explanation
explanation.show_in_notebook()  # Or print(explanation.as_list())

In [ ]:
import torch
import matplotlib.pyplot as plt

# Select a test comment and tokenize
test_comment = test_dataset[0]["input_text"]
inputs = tokenizer(test_comment, return_tensors="pt").to(model.device)

# Get attention weights
model.eval()
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)
    attentions = outputs.attentions  # List of attention matrices per layer

# Average attention weights across heads and layers (simplified)
avg_attention = torch.stack(attentions).mean(dim=0).mean(dim=1)[0].cpu().numpy()

# Get token labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Plot attention heatmap
plt.figure(figsize=(10, 8))
plt.imshow(avg_attention, cmap="viridis")
plt.xticks(range(len(tokens)), tokens, rotation=45)
plt.yticks(range(len(tokens)), tokens)
plt.colorbar()
plt.title("Attention Heatmap for Comment")
plt.show()

In [ ]:
from transformers import pipeline

In [ ]:
# Load your model and tokenizer
generator = pipeline("text-generation", model="your_model_name_here")

# Define test prompts
prompts = [
    # Basic Functional Prompts
    "Comment: Blink an LED connected to pin PA5 at 1 Hz",
    "Comment: Configure SPI1 in master mode",
    "Comment: Send the string 'Hello' over USART2",
    "Comment: Set GPIO pin PB3 as output",

    # Detailed/Explanatory Prompts
    "Comment: Implement a function that waits for 2 seconds using HAL delay",
    "Comment: Initialize the ADC1 peripheral and start a conversion",
    "Comment: Create a function to toggle LED2 every 250ms",
    "Comment: Configure the system clock using HSE with PLL",

    # Action-Oriented Prompts
    "Comment: Toggle the onboard LED",
    "Comment: Start PWM on TIM2 Channel 1",
    "Comment: Reset the watchdog timer",
    "Comment: Enable interrupts for EXTI line 13",

    # Structured/Formal Prompts
    "Comment: void Init_I2C1(void) — initialize I2C1 with 100kHz standard mode",
    "Comment: void Transmit_UART1(char* data) — send a null-terminated string over UART1",
    "Comment: void GPIO_SetPin(GPIO_TypeDef* GPIOx, uint16_t GPIO_Pin) — set a GPIO pin high",

    # Error/Edge Case Prompts
    "Comment: Try to write to a NULL pointer",
    "Comment: Handle timeout if I2C transmission fails",
    "Comment: Return an error if UART buffer is full",
]

# Function to test each prompt
def test_prompts(prompts):
    for prompt in prompts:
        print(f"Testing prompt: {prompt}\n")
        response = generator(prompt, max_length=200, num_return_sequences=1)
        print(f"Generated Code: {response[0]['generated_text']}\n")
        print("="*80)

# Run tests
test_prompts(prompts)